In [ ]:
import os
from pathlib import Path
from shutil import copyfile
import re 
import platform
import pickle
import numpy as np
import pandas as pd
import re


In [ ]:


def clean_title(input_name):
    # first do generic cleaning
    input_name = input_name.split('_')[0]
    input_name = input_name.split('(')[0]
    input_name = input_name.split('[')[0]
    
    # if name is too short after split, don't do this
    if (not len(input_name.split(':')[0].split())==1):
        input_name = input_name.split(':')[0]
    
    input_name = input_name.strip()
    # THEN apply hard coded rule
    if input_name in dict_subs:
        input_name = dict_subs[input_name]
    return input_name

In [ ]:
copy_lib = True
calibre_library_path = '/mnt/c/Users/Thiago/Calibre - Open Culture/'

In [ ]:
from pathlib import Path

In [ ]:
dir_books = Path(Path.cwd(),'books_raw')
if not os.path.exists(dir_books):
    os.makedirs(dir_books)

In [ ]:

if copy_lib:

    root = calibre_library_path

    pattern = re.compile('^.*\((.*)\)$')

    for path, subdirs, files in os.walk(root):
        for name in files:
            if(Path(name).suffix=='.txt'):
                path_origin = os.path.join(path, name)

                p = Path(path_origin).parts[-2]
                id_book = re.match(pattern,p)[1]            
                # lets work with raw file names, clean name on metadata!
                # clean_name = clean_title(clean_name)


                path_dest = os.path.join(dir_books,id_book + '.txt')
                
                if not(os.path.isfile(f'{dir_books}/{id_book}.txt')):
                    copyfile(path_origin, path_dest)


In [ ]:
loc_path = str(Path.cwd())

In [ ]:
loc_path

### Read Metadata from my books

In [ ]:
metadata_calibre = pd.read_csv('My books.csv')
metadata_calibre = metadata_calibre.set_index('id')

In [ ]:
# delete books not found from metadata for the rest of the analysis
for id_book in metadata_calibre.index:
    
    if not os.path.isfile(f'{loc_path}/books_raw/{id_book}.txt'):
        print(f'{loc_path}/books_raw/{id_book}.txt')
        metadata_calibre = metadata_calibre.drop(metadata_calibre.query(f"id=={id_book}").index)

### HARD CODE CORRECTIONS TO TITLE -> CREATE CLEAN_TITLES

In [ ]:
dict_subs = {
    'ULYSSES ' : "Ulysses",
    'The Poetics. Translated From Greek Into English and From Arabic Into Latin' : "Aristotle's Poetics",
    'War and Peace by Leo Tolstoy' : 'War and Peace',
    'Douglas Adams, The Restaurant at the End of the Universe #2 ' : "The Restaurant at the End of the Universe",
    'CliffsComplete King Lear' : "King Lear",
    'Kant': "Kant Complete Works",
    'Bach' : "Music in the Castle of Heaven",
    'Oscar Wilde' : "Oscars Wilde Complete Works",
    'Plato' : 'Plato Five Dialogues',
    'tmp4C9D' : "The Writer's Journey",
    'Islam' : "Islam: A Short Story",
    'Story' : 'Story: Style, Structure, Substance',
    'Delphi Collected Works of Arthur Schopenhauer' : "The Collected Works of Arthur Schopenhauer",
    'Justice' : "Justice: What's the Right Thing to do?",
    'Hamlet, Prince of Denmark' : "Hamlet",
    'The Dream of a Ridiculous Man Annotated and Illustrated Book' : 'The Dream of a Ridiculous Man',
    'Collected Fictions' : "Borges Fictions"
}

In [ ]:


# create clean title column!!!
metadata_calibre['clean_title'] = metadata_calibre['title'].apply(clean_title)


### ADD ALIASES
### DONT CORRECT TYPOS HERE

In [ ]:
use_fantasy = False

names = []
metadata_aliases = {}

for id_,stem in zip(metadata_calibre.index,metadata_calibre['clean_title']):
                if(stem=='Keats Complete Works'):
                    metadata_aliases.update({id_:["Keat's Hyperion"]})
                elif(stem=='Kant Complete Works'):
                     metadata_aliases.update({id_: ['Critique of Pure Reason']})
                elif(stem=='Oscar Wilde Complete Works'):
                     metadata_aliases.update({id_: ['The Importance of Being Earnest','Salomé']})
                elif(stem=='The Collected Works of Arthur Schopenhauer'):
                     metadata_aliases.update({id_: ['The World As Will And Idea']})
                elif(stem=='Plato Five Dialogues'):
                    metadata_aliases.update({id_: 
                                  [
                                        'Euthyphro',
                                        "Socrate's Apology",
                                        'Crito',
                                        'Meno',
                                        'Phaedo'
                                  ]})
                elif(stem=='Ulysses'):
                    metadata_aliases.update({id_: ["Joyce's Ulysses"]})
                elif(stem=='Borges Fictions'):
                     metadata_aliases.update({id_:
                                 
                                 [
                                        'Tlön, Uqbar, Orbis Tertius',
                                        'Pierre Menard, Author of the Quixote',
                                        'The Circular Ruins',
                                        'The Lottery in Babylon',
                                        'A Survey of the Works of Herbert Quain',
                                        'The Library of Babel',
                                        'The Garden of Forking Paths',
                                        'The Aleph',
                                 ]})
                elif(stem=='Dune'):
                     metadata_aliases.update({id_: ["Dune"]})
                elif(stem=='The Republic of Plato'):
                     metadata_aliases.update({id_: ["Plato's Republic","The Republic"]})
                elif(stem=='Meditations'):
                     metadata_aliases.update({id_: ["Marcus Aurelius's Meditations"]})
                elif(stem=='The Complete Essays'):
                     metadata_aliases.update({id_: ["Montaigne's Essays"]})
                elif(stem=='Ethics'):
                     metadata_aliases.update({id_: ["Spinoza's Ethics"]})
                else:
                     if (len(stem.split())!=1):   
                        metadata_aliases.update({id_:[stem]})
                     else:
                        metadata_aliases.update({id_:[f'{stem}']})

In [ ]:
# remove goddamn The Dispossed
try:
    metadata_calibre = metadata_calibre.drop(index=20)
except:
    pass

In [ ]:
# add aliases to metadata
metadata_calibre['aliases'] = pd.Series(metadata_aliases)

## Exclude books from analysis by name!
## MY books
#### e.g. dictionaries, fantasy books etc, books not in english

In [ ]:
# languages
metadata_calibre = metadata_calibre.query('languages=="eng"')

# no dictionaries
metadata_calibre = metadata_calibre[~metadata_calibre["title"].str.contains('ictionary', na=False)]

# no poetry
metadata_calibre = metadata_calibre[~metadata_calibre["title"].str.contains('Poems', na=False)]


# take modern fantasy out 
list_fantasy_writers = [
                        'Brandon Sanderson',
                        'Patrick Rothfuss',
                        'Jim Butcher',
                        'Brent Weeks',
                        'Robin Hobb',
                        'Joe Abercrombie',
                        'Robert Jordan',
                        'Jim Butcher & Mark Powers',
                        'Stephen King',
                        'Terry Pratchett',
                        'Neil Gaiman & Terry Pratchett',
                        'Jo Nesbø',
                       ]

metadata_calibre = metadata_calibre[~metadata_calibre["authors"].isin(list_fantasy_writers)]


### Export some metadata to make a table on webpage

In [ ]:

to_export = ['clean_title','authors','pubdate']
metadata_calibre[to_export].to_csv('web/book_metadata.csv')

### Hardcode some books to not look for citations inside then

In [ ]:

# these don't cite other books
# so we just use them to point other citations
excluded_books = ['King Lear','Hamlet','The Tempest','Othello','The Odyssey','Ulysses']

## Construct Graph of Citations between my books

In [ ]:
#def find_word(s,w):
#    return re.search(r'(\b|,){}(\b|[?!.,])'.format(w), s)
# in awk
# awk '/(\s|,)Nietzsche(\s|[?!.,])/{print NR}' 135.txt

### Make big REGEX

In [ ]:


# humongous regex 
list_words = []
for id_book,aliases_target_book in zip(metadata_calibre.index,metadata_calibre['aliases']):
    for book in aliases_target_book:
        list_words.append(book)
        

regex = '[, ]('
for word in list_words:
    word = re.escape(word)
    regex+=f'{word}|'
regex = regex[:-1]
regex+=')[?!., ]'

humongous_regex = re.compile(regex)

def find_any_book(s):
    return re.findall(humongous_regex, s)

In [ ]:
find_any_book('Eu gosto de , Hamlet, ??')


In [ ]:
import pickle

os.makedirs(loc_path + '/pickled_metadata')

pickle.dump( metadata_calibre, open( "pickled_metadata/calibre_metadata.p", "wb" ) )
pickle.dump( humongous_regex, open( "pickled_metadata/calibre_regex.p", "wb" ) )

## Read Goodreads data and Prepare


In [ ]:

df_orig = pd.read_csv('goodreads_ds.csv',error_bad_lines=False)


metadata_goodreads = df_orig[df_orig['language_code']=='eng']
metadata_goodreads['clean_title'] = metadata_goodreads['title'].apply(clean_title)
metadata_goodreads['num_words_title'] = metadata_goodreads['clean_title'].apply(lambda x: len(x.split()))


# remove some random unknown books
metadata_goodreads = metadata_goodreads[metadata_goodreads['ratings_count']>200]

### just to make dataset
#df = df[df['num_words_title']<=2]
metadata_goodreads['len_clean_title'] = metadata_goodreads['clean_title'].apply(len)
metadata_goodreads = metadata_goodreads[metadata_goodreads['len_clean_title']>5]

metadata_goodreads = metadata_goodreads.drop_duplicates(subset=['clean_title'])



### Use following unigram data to filter book titles that are too common words like "Divine" or "Cathedral"

In [ ]:
## load frequency dataset 
freq_ds = pd.read_csv('unigram_freq.csv')


In [ ]:
def get_ranking(book):
    try:
        return int(freq_ds.query(f'word=="{book.lower()}"').index.values[0])
    except: 
        return 100000000000


metadata_goodreads['word_freq_on_english'] = metadata_goodreads['clean_title'].apply(get_ranking)
metadata_calibre['word_freq_on_english'] = metadata_calibre['clean_title'].apply(get_ranking)



In [ ]:
too_common_word_titles = list(metadata_goodreads.sort_values(by='word_freq_on_english')['clean_title'].head(20).values)

In [ ]:
for word in too_common_word_titles:
    print(word,metadata_goodreads.query(f'clean_title=="{word}"')[['title','year_first_published','word_freq_on_english']].values)

In [ ]:
# filter by books whose names are too common on the english language as words

metadata_goodreads = metadata_goodreads[metadata_goodreads['word_freq_on_english']>3000]

In [ ]:
metadata_goodreads[['clean_title','title','word_freq_on_english']].to_csv('word_freq_goodreads.csv')

In [ ]:
metadata_calibre[['clean_title','title','word_freq_on_english']].to_csv('word_freq_calibre.csv')

In [ ]:
bad_book_list = ['Divine','Caesar','Silence','Cathedral']
bad_book_list.extend(too_common_word_titles)

for bad_book in bad_book_list:
    
    metadata_goodreads= metadata_goodreads.drop(index=metadata_goodreads.query(f'clean_title=="{bad_book}"').index)

In [ ]:
metadata_aliases = {}

for id_,stem in zip(metadata_goodreads.index,metadata_goodreads['clean_title']):
                     if (len(stem.split())!=1):   
                        metadata_aliases.update({id_:[stem]})
                     else:
                        metadata_aliases.update({id_:[f'{stem}']})

In [ ]:
metadata_goodreads['aliases'] = pd.Series(metadata_aliases)

In [ ]:
# sort by descending number of words so books like Journey are not matched before Writer's Journey!
metadata_goodreads = metadata_goodreads.sort_values(by='num_words_title', ascending=False)

### remove books from goodreads ds that are already on my list

In [ ]:
for id_book,aliases_target_book in zip(metadata_calibre.index,metadata_calibre['aliases']):
    for book in aliases_target_book:
        if book in list(metadata_goodreads.clean_title):
            metadata_goodreads = metadata_goodreads.drop(index=metadata_goodreads.query(f'clean_title=="{book}"').index)

## Add citations from my books to goodreads books to graph

### Regex for Goodreads books

In [ ]:
import re

In [ ]:
# humongous regex for goodreads
list_words_goodreads = []
for id_book,aliases_target_book in zip(metadata_goodreads.bookID,metadata_goodreads['aliases']):
    for book in aliases_target_book:
        list_words_goodreads.append(book)
        
list_words_goodreads = list(set(list_words_goodreads))        

regex = '[, ]('
for word in list_words_goodreads:
    word = re.escape(word)
    regex+=f'{word}|'
regex = regex[:-1]
regex+=')[?!., ]'

humongous_regex_goodreads = re.compile(regex)

def find_any_book_goodreads(s):
    return re.findall(humongous_regex_goodreads, s)

In [ ]:
"The Story of Philosophy" in list_words_goodreads

In [ ]:
pickle.dump( metadata_goodreads, open( "pickled_metadata/metadata_goodreads.p", "wb" ) )
pickle.dump( humongous_regex_goodreads, open( "pickled_metadata/goodreads_regex.p", "wb" ) )